In [ ]:
%pip install google-cloud-translate
%pip install requests pandas tqdm

In [2]:
import pandas as pd
import requests
import os
from tqdm.auto import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
from transformers import AutoTokenizer

# --- Configuration ---
API_KEY = 'API_KEY'

# Summary Translations

In [ ]:
MAX_WORKERS = 15

# The Google Translate API endpoint
TRANSLATE_ENDPOINT = 'https://translation.googleapis.com/language/translate/v2'

def translate_text_with_key(text, target_language):
    """
    Translates a single piece of text using the Google Translate API.
    This function will be executed by each worker thread.
    """
    if not isinstance(text, str) or not text.strip():
        return ""

    payload = {
        'q': text,
        'target': target_language,
        'key': API_KEY,
        'format': 'text'
    }
    
    try:
        response = requests.post(TRANSLATE_ENDPOINT, data=payload)
        response.raise_for_status()
        result = response.json()
        translated_text = result['data']['translations'][0]['translatedText']
        # 100ms delay to avoid hitting rate limits
        time.sleep(0.1)
        return translated_text
    except requests.exceptions.RequestException as e:
        # This error is now more important in a parallel run
        # We return the original text to avoid data loss
        print(f"HTTP Error for text '{text[:30]}...': {e}. Returning original text.")
        return text
    except Exception as e:
        print(f"An unexpected error occurred for text '{text[:30]}...': {e}. Returning original text.")
        return text

def parallel_translate_column(series: pd.Series, target_language: str) -> list:
    """
    Translates a pandas Series in parallel using a ThreadPoolExecutor.
    
    Args:
        series (pd.Series): The column of text to translate.
        target_language (str): The target language code (e.g., 'en', 'bn').
        
    Returns:
        list: A list of translated texts in the original order.
    """
    # We need to store results in a way that preserves the original order
    results = [None] * len(series)
    
    # Use ThreadPoolExecutor to manage a pool of threads
    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        # Create a dictionary to map future objects to their original index
        future_to_index = {executor.submit(translate_text_with_key, text, target_language): i for i, text in enumerate(series)}
        
        # Use tqdm to create a progress bar as futures complete
        progress_bar = tqdm(as_completed(future_to_index), total=len(series), desc=f"Translating to {target_language}")
        
        for future in progress_bar:
            index = future_to_index[future]
            try:
                # Get the result from the future
                results[index] = future.result()
            except Exception as e:
                # If a future itself fails (though errors are caught inside the function)
                print(f"A task failed at index {index}: {e}")
                results[index] = series.iloc[index] # Return original text on failure

    return results


# --- Load and Prepare Data ---
input_path = '/home/server02/mahmud/mt/outputs/articles.csv'
try:
    df = pd.read_csv(input_path)
    print(f"Successfully loaded data from {input_path}")
except FileNotFoundError:
    print(f"Error: The file was not found at {input_path}")
    exit()

df = df.dropna(subset=['summary', 'summary_en'])

print(f"Processing {len(df)} rows using up to {MAX_WORKERS} parallel workers.")

# --- Perform Translations in Parallel ---

# Translate Bengali ('summary') to English
df['summary_en_gt'] = parallel_translate_column(df['summary'], 'en')

# Translate English ('summary_en') to Bengali
df['summary_gt'] = parallel_translate_column(df['summary_en'], 'bn')


# --- Save the Results ---
output_path = '/home/server02/mahmud/mt/outputs/summaries_final.csv'
df.to_csv(output_path, index=False, encoding='utf-8')

print(f"\nParallel translation complete!")
print(f"The DataFrame with translated columns has been saved to: {output_path}")

# Display the first few rows of the final DataFrame to verify
print("\nPreview of the final DataFrame:")
display(df.head())

In [3]:
input_path = '/home/server02/mahmud/mt/outputs/summaries_final.csv'
df = pd.read_csv(input_path)
df

,id,source_name,content,content_en,summary,summary_en,summary_en_gt,summary_gt,rouge_1_f,rouge_l_f,rouge_avg
0,26608,samakal,ক্লাব বিশ্বকাপে মেক্সিকান ক্লাব পাচুকার বিপক্ষ...,Real Madrid won 3-1 against Mexican club Pachu...,ক্লাব বিশ্বকাপে মেক্সিকোর ক্লাব পাচুকাকে ৩-১ গ...,Real Madrid defeated Mexican club Pachuca 3-1 ...,Real Madrid beat Mexican club Pachuca 3-1 in t...,ক্লাব বিশ্বকাপে জার্মান ডিফেন্ডার আন্তোইন রুডি...,0.380952,0.304762,0.342857
1,26604,samakal,টানা ৪০ দিন বন্ধ থাকার পর অবশেষে চালু হলো ঢাকা...,"After being closed for 40 consecutive days, Dh...",৪০ দিন বন্ধ থাকার পর আজ সোমবার ঢাকা দক্ষিণ সিট...,"Dhaka South City Corporation's headquarters, c...","After being closed for 40 days, the Dhaka Sout...",৪০ দিন ধরে বন্ধ ঢাকা দক্ষিণ সিটি কর্পোরেশনের স...,0.419580,0.335664,0.377622
2,26599,samakal,জাতীয় রাজস্ব বোর্ডের-এনবিআর পাঁচ উপ-কর কমিশনা...,Five deputy tax commissioners of the National ...,জাতীয় রাজস্ব বোর্ডের (এনবিআর) পাঁচ উপকর কমিশন...,Five deputy tax commissioners of the National ...,Five Excise Commissioners of the National Boar...,জাতীয় রাজস্ব বোর্ডের (এনবিআর) পাঁচজন উপ-কর কম...,0.421053,0.359133,0.390093
3,26598,samakal,সাবেক প্রধান নির্বাচন কমিশনার হাবিবুল আউয়ালকে...,Former Chief Election Commissioner Habibul Awa...,সাবেক প্রধান নির্বাচন কমিশনার (সিইসি) মো. নূরু...,Former Chief Election Commissioner Habibul Awa...,Former Chief Election Commissioner (CEC) Md. N...,স্বরাষ্ট্র উপদেষ্টা লেফটেন্যান্ট জেনারেল জাহাঙ...,0.462963,0.444444,0.453704
4,26596,samakal,সিরিয়ার উত্তরপূর্বাঞ্চলীয় হাসাকাহ প্রদেশে অব...,A US military base in the northeastern Syrian ...,সিরিয়ার হাসাকা প্রদেশে মার্কিন সামরিক ঘাঁটিতে...,A US military base in Hasakah province Syria w...,A mortar attack has hit a US military base in ...,"ইরানের সংবাদমাধ্যম মেহের নিউজ জানিয়েছে, সিরিয...",0.426966,0.382022,0.404494
...,...,...,...,...,...,...,...,...,...,...,...
1498,18490,samakal,বিভিন্ন জালিয়াতিতে সম্পৃক্ততার দায়ে ইসলামী ব...,Islamic Bank Bangladesh's controversial MD Muh...,জালিয়াতির অভিযোগে ইসলামী ব্যাংক বাংলাদেশের ব্...,Islamic Bank Bangladesh's Managing Director Mu...,"Muhammad Munirul Mawla, Managing Director of I...",বাংলাদেশ ব্যাংকের বোর্ডের সিদ্ধান্তের পর জালিয...,0.397516,0.254658,0.326087
1499,18488,samakal,শাহ্জালাল ইসলামী ব্যাংক পিএলসি’র রিস্ক ম্যানেজ...,The 70th meeting of the Risk Management Commit...,শাহজালাল ইসলামী ব্যাংক পিএলসির ৭০তম রিস্ক ম্যা...,The 70th Risk Management Committee meeting of ...,The 70th Risk Management Committee meeting of ...,শাহজালাল ইসলামী ব্যাংক পিএলসি-এর ৭০তম ঝুঁকি ব্...,0.620000,0.560000,0.590000
1500,18480,samakal,ভারতের ছত্তিশগড় রাজ্যের নারায়ণপুর ও দান্তেওয...,At least 30 Maoists were killed in an operatio...,ভারতের ছত্তিশগড়ের পুর জেলায় নিরাপত্তা বাহিনী...,At least 30 Maoists were killed in a security ...,At least 30 Maoists have been killed in a secu...,ছত্তিশগড়ের পুর জেলায় নিরাপত্তা বাহিনীর অভিযা...,0.371541,0.276680,0.324111
1501,18476,samakal,পাকিস্তানে টি-২০ সিরিজ খেলতে যাবে বাংলাদেশ জাত...,Bangladesh national cricket team will tour Pak...,"লাহোরের গাদ্দাফি স্টেডিয়ামে ২৮, ৩০ ও ১ জুন শু...",The Bangladesh national cricket team will tour...,The Bangladesh national cricket team will trav...,"বাংলাদেশ জাতীয় ক্রিকেট দল ২৮, ৩০ মে এবং ১ জুন...",0.427861,0.278607,0.353234


# Article Translations

In [ ]:
MAX_WORKERS = 15

# The Google Translate API endpoint
TRANSLATE_ENDPOINT = 'https://translation.googleapis.com/language/translate/v2'

def translate_text_with_key(text, target_language):
    """
    Translates a single piece of text using the Google Translate API.
    This function will be executed by each worker thread.
    """
    if not isinstance(text, str) or not text.strip():
        return ""

    payload = {
        'q': text,
        'target': target_language,
        'key': API_KEY,
        'format': 'text'
    }
    
    try:
        response = requests.post(TRANSLATE_ENDPOINT, data=payload)
        response.raise_for_status()
        result = response.json()
        translated_text = result['data']['translations'][0]['translatedText']
        return translated_text
    except requests.exceptions.RequestException as e:
        # This error is now more important in a parallel run
        # We return the original text to avoid data loss
        print(f"HTTP Error for text '{text[:30]}...': {e}. Returning original text.")
        return text 
    except Exception as e:
        print(f"An unexpected error occurred for text '{text[:30]}...': {e}. Returning original text.")
        return text

def parallel_translate_column(series: pd.Series, target_language: str) -> list:
    """
    Translates a pandas Series in parallel using a ThreadPoolExecutor.
    
    Args:
        series (pd.Series): The column of text to translate.
        target_language (str): The target language code (e.g., 'en', 'bn').
        
    Returns:
        list: A list of translated texts in the original order.
    """
    # We need to store results in a way that preserves the original order
    results = [None] * len(series)
    
    # Use ThreadPoolExecutor to manage a pool of threads
    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        # Create a dictionary to map future objects to their original index
        future_to_index = {executor.submit(translate_text_with_key, text, target_language): i for i, text in enumerate(series)}
        
        # Use tqdm to create a progress bar as futures complete
        progress_bar = tqdm(as_completed(future_to_index), total=len(series), desc=f"Translating to {target_language}")
        
        for future in progress_bar:
            index = future_to_index[future]
            try:
                # Get the result from the future
                results[index] = future.result()
            except Exception as e:
                # If a future itself fails (though errors are caught inside the function)
                print(f"A task failed at index {index}: {e}")
                results[index] = series.iloc[index] # Return original text on failure

    return results


# --- Load and Prepare Data ---
input_path = '/home/server02/mahmud/mt/outputs/articles_final.csv'
try:
    df = pd.read_csv(input_path)
    print(f"Successfully loaded data from {input_path}")
except FileNotFoundError:
    print(f"Error: The file was not found at {input_path}")
    exit()

df = df.dropna(subset=['textbn', 'texten'])

print(f"Processing {len(df)} rows using up to {MAX_WORKERS} parallel workers.")

# --- Perform Translations in Parallel ---

# Translate Bengali('textbn') to English
df['texten_gt'] = parallel_translate_column(df['textbn'], 'en')

# Translate English('texten') to Bengali
df['textbn_gt'] = parallel_translate_column(df['texten'], 'bn')


# --- Save the Results ---
output_path = '/home/server02/mahmud/mt/outputs/articles_final.csv'
df.to_csv(output_path, index=False, encoding='utf-8')

print(f"\nParallel translation complete!")
print(f"The DataFrame with translated columns has been saved to: {output_path}")

# Display the first few rows of the final DataFrame to verify
print("\nPreview of the final DataFrame:")
display(df.head())

In [4]:
input_path = '/home/server02/mahmud/mt/outputs/articles_final.csv'
df = pd.read_csv(input_path)
df

,textbn,texten,textbn_predicted,texten_predicted,texten_gt,textbn_gt
0,"ট্র্যাফিক জ্যামে আটকে যখনহচ্ছে বাসে বাদুরঝোলা,...","Stuck in a traffic jam, when the Badurjola bus...",যানজটে আটকা পড়লে বাদুরঝোলা বাস থামলে চোখ খোলা...,"When a bus is stuck in traffic jams, you will ...",When you are stuck in a traffic jam and hangin...,"যানজটে আটকে থাকা অবস্থায়, বাদুরজলা বাস থামলে,..."
1,সশস্ত্রবাহিনী আয়োজিত ইফতার মাহফিলে অংশ নিয়েছ...,Prime Minister Sheikh Hasina attended an Iftar...,সশস্ত্র বাহিনী আয়োজিত ইফতার পার্টিতে যোগ দিয়...,Prime Minister Sheikh Hasina attended an Iftar...,Prime Minister Sheikh Hasina has participated ...,প্রধানমন্ত্রী শেখ হাসিনা সশস্ত্র বাহিনী আয়োজি...
2,স্বাধীনতার স্থপতি বঙ্গবন্ধু শেখ মুজিবুর রহমানে...,The book fair on the occasion of Father of the...,জাতির পিতা বঙ্গবন্ধু শেখ মুজিবুর রহমানের জন্মব...,A book fair has begun at Shishu Academy on the...,A book fair has started at Shishu Academy on t...,জাতির পিতা বঙ্গবন্ধু শেখ মুজিবুর রহমানের জন্মব...
3,বিএনপির নতুন মহাসচিব মির্জা ফখরুল ইসলাম আলমগীর...,BNP's new secretary general Mirza Fakhrul Isla...,বিএনপির নতুন মহাসচিব মির্জা ফখরুল ইসলাম আলমগীর...,BNP's new secretary general Mirza Fakhrul Isla...,BNP's new Secretary General Mirza Fakhrul Isla...,বিএনপির নতুন মহাসচিব মির্জা ফখরুল ইসলাম আলমগীর...
4,কোনো কারণ ছাড়াই নিউজ প্রোটাল পরিবর্তন ডটকম বন...,The Surokkha Forum leaders have expressed deep...,কোনো কারণ ছাড়াই নিউজ পোর্টাল পরিবর্তন ডটকম বন...,"The leaders of the Sushil Forum, including Pre...","Sushil Forum President Md. Zahid, Senior Vice ...",সুরক্ষা ফোরামের নেতারা কোনও কারণ ছাড়াই নিউজ প...
...,...,...,...,...,...,...
4911,আফগানিস্তানের মধ্যাঞ্চলীয় শহর গজনি দখলে নিতে ...,The Taliban attacked government forces on Tues...,আফগানিস্তানের মধ্যাঞ্চলীয় শহর গজনি দখলে নিতে ...,The Taliban have launched an offensive against...,The Taliban have launched an attack on governm...,মঙ্গলবার তালেবানরা মধ্য আফগানিস্তানের গজনি শহর...
4912,বিজ্ঞ বিচারকদের উপস্থিতিতে গত ২৮ এপ্রিল হয়ে গ...,"In the presence of eminent judges, the countin...",বিশিষ্ট বিচারকদের উপস্থিতিতে গত ২৮ এপ্রিল মেরি...,"In the presence of eminent judges, the countin...",The counting of votes cast by readers in the M...,"বিশিষ্ট বিচারকদের উপস্থিতিতে, মেরিল-প্রথম আলো ..."
4913,আগামী ১০ মার্চের মধ্যে কভিড-১৯ টিকা গ্রহণের নি...,All the teachers of the country have been inst...,দেশের সব শিক্ষককে আগামী ১০ মার্চের মধ্যে কোভিড...,All teachers in the country have been instruct...,All teachers in the country are instructed to ...,দেশের সকল শিক্ষককে ১০ মার্চের মধ্যে কোভিড-১৯ ট...
4914,প্রথমবারের মতো একটি দ্বৈত গান নিয়ে সংগীত শিল্...,Music artists Shah Hamza and popular artist Na...,প্রথমবারের মতো একটি দ্বৈত গানে এক হলেন সংগীতশি...,"For the first time, music artist Shah Hamza an...","For the first time, music artist Shah Hamza an...",সঙ্গীত শিল্পী শাহ হামজা এবং জনপ্রিয় শিল্পী ন্...
